In [6]:
import numpy as np

# generate some linear-looking data
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

In [7]:
# compute theta using the normal equation
X_b = np.c_[np.ones((100, 1)), X]  # add x0 = 1 to each instance
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

In [8]:
# print values of theta 0 and theta 1
theta_best

array([[4.1408547 ],
       [3.02130891]])

# Linear Regression

In [10]:
# import the model
from sklearn.linear_model import LinearRegression

# initialize the model
lin_reg = LinearRegression()

# fit the model
lin_reg.fit(X,y)

# print intercept and coefficient
lin_reg.intercept_, lin_reg.coef_

(array([4.1408547]), array([[3.02130891]]))

In [11]:
# least squares
theta_best_svd, residuals, rank, s = np.linalg.lstsq(X_b, y, rcond=1e-6)

In [12]:
theta_best_svd

array([[4.1408547 ],
       [3.02130891]])

In [13]:
# compute the psuedoinverse
np.linalg.pinv(X_b).dot(y)


array([[4.1408547 ],
       [3.02130891]])

Important note: the Singular Value Decomposition (SVD) approach used by sklearn's linear regression class is about O(n sqaured). Be warned that when the number of features doubles, the computation time increases by roughly four times.

Both the normal equation and SVD get very slow when the number of features is large (100k), however, they are both linear with regard to number of instances in the training set so they can handle large training sets as long as those sets can fit in memory.

# Gradient Descent

definition: "a generic optimizatino algorithm capable of finding optimal solutions to a wide range of problems"

basic idea is to adjust parameters iteratively in order to minimize a cost function

learning rate matters a lot - to slow and you never arrive at the optimal theta, too large and you jump back and forth across the valley and never get to the bottom

goal: get to the global minimum

NOTE: gradient descent requires that all features have a similar scale (e.g. StandardScaler) or else it will take too long to converge

# Batch Gradient Descent


Something to remember, one of the challenges with batch gradient descent is that it uses the whole training set to compute the gradients at every step which makes it very slow.

In [15]:
eta = 0.1  # learning rate
n_iterations = 1000
m = 100

theta = np.random.randn(2,1)  # random initialization

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients

In [16]:
theta

array([[4.1408547 ],
       [3.02130891]])

# Stochastic Gradient Descent

The helpful thing about stochastic gradient descent is that it picks a random instance in the training set at every step and only computes based on that instance. There's less data to manipulate and it is much faster. Which also makes it better suited for large training sets. It can also be implemented out-of-core.

In [17]:
n_epochs = 50
t0, t1 = 5, 50  # learning schedule hyperparameters

def learning_schedule(t):
    return t0 / (t + t1)

theta = np.random.randn(2,1)  # random initialization

for epoch in range(n_epochs):
    for i in range(m):
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta * gradients

In [18]:
theta

array([[4.17308617],
       [3.01853635]])

The upside of randomness is that you have a better chance of making progress and the downside is that you can never settle at the global minimum. One way to counteract this is to start with large learning steps and them slowly get smaller ad smaller.

In [19]:
# here's another example with a learning schedule:
n_epochs = 50
t0, t1 = 5, 50  # learning schedule hyperparameters

def learning_schedule(t):
    return t0 / (t + t1)

theta = np.random.randn(2,1)  # random initialization

for epoch in range(n_epochs):
    for i in range(m):
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta * gradients

In [20]:
theta

array([[4.17870878],
       [2.97553957]])

fun fact: each iteration is called an "epoch"

In [21]:
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1)
sgd_reg.fit(X, y.ravel())

SGDRegressor(eta0=0.1, penalty=None)

In [22]:
sgd_reg.intercept_, sgd_reg.coef_

(array([4.2058649]), array([3.10548244]))